# TP N° 1: Análisis Exploratorio de Datos
## Heart Diseases Dataset
Este conjunto de datos es una versión avanzada del clásico conjunto de datos de enfermedades cardíacas de UCI Machine Learning, enriquecido con más características para soportar análisis más sofisticados.

## 1- Listado de variables y selección 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import h5py
import PIL
import seaborn as sns
import plotly
import sklearn_pandas
import plotly.express as px

In [ ]:
data = pd.read_csv('./heart_disease_data_with_features.csv')

In [ ]:
data.shape
# Muestra las dimensiones del dataframe

In [ ]:
data.head(10)

In [ ]:
# Dividimos por las columnas de interes
subset = data.loc[:, ['sex','cp','fbs','restecg','thalach', 'exang','oldpeak','slope', 'ca','thal', 'num','age_group', 'cholesterol_level', 'bp_level','risk_score', 'symptom_severity', 'risk_factor', 'avg_chol_by_age_group']]
subset.describe()

In [ ]:
subset.head(10)

In [ ]:
# Mostramos los tipos de datos
subset.dtypes

In [ ]:
# Elije 10 filas al azar del DataFrame
subset.sample(10)

In [ ]:
fig = px.box(subset, y='thalach', )
fig.show()

Analizando la variable "thalach" podemos ver que uno de los registros que tiene es atípico. Este registro va a ser eliminado porque puede perjudicar el modelo. 

In [ ]:
result = subset[subset['thalach'] == 71]
result.head()

In [ ]:
subset = subset.drop(index=245)

In [ ]:
fig = px.box(subset, y='thalach', )
fig.show()

Con el gráfico en barras podemos deducir que en la mayoria de datos del subconjunto es 0, es decir, que no se detectaron enfermedades. Y luego la cantidad de casos donde se empieza a detectar enfermedades se encuentran escalonadas, yendo desde el 1 (enfermedad leve) hasta 4 (enfermedad grave).

## 2- Análisis detallado de un conjunto de variables

### Variables Nulos
Estos son los valores nulos encontrados en el subset de datos:

In [ ]:
subset.isnull().sum()

#### Tratamiento de valores nulos

**Variables:**
- **ca: numerica**
- **thal: numerica**
- **age_group: cualitativa**
- **cholesterol_level: cualitativa**
- **bp_level: cualitativa**
- **risk_factor: numerica**
- **avg_chol_by_age_group: numerica**

Para las variables numericas, el tratamiento que llevaremos a cabo sera rellenar con la media
Para las variables cualitativas, el tratamiento será rellenar con el valor que mas se repite

#### tratamiento de valores nulos de 'ca'

In [ ]:
subset['ca'] = subset['ca'].fillna(subset['ca'].mean())
subset['ca'].isnull().sum()

#### tratamiento de valores nulos de 'thal'

In [ ]:
subset[subset['thal'].isna()]

In [ ]:
subset['thal'] = subset['thal'].fillna(subset['thal'].mean())
subset['thal'].isnull().sum()

#### tratamiento de valores nulos de 'age_group'

In [ ]:
subset['age_group'] = subset['age_group'].fillna(subset['age_group'].mode()[0])
subset['age_group'].isnull().sum()

#### tratamiento de valores nulos de 'cholesterol level'

In [ ]:
subset['cholesterol_level'] = subset['cholesterol_level'].fillna(subset['cholesterol_level'].mode()[0])
subset['cholesterol_level'].isnull().sum()

#### tratamiento de valores nulos de 'bp_level'

In [ ]:
subset['bp_level'] = subset['bp_level'].fillna(subset['bp_level'].mode()[0])
subset['bp_level'].isnull().sum()

#### tratamiento de valores nulos de 'risk_factor'

In [ ]:
subset['risk_factor'] = subset['risk_factor'].fillna(subset['risk_factor'].mean())
subset['risk_factor'].isnull().sum()

#### tratamiento de valores nulos de 'avg_chol_by_age_group'

In [ ]:
subset['avg_chol_by_age_group'] = subset['avg_chol_by_age_group'].fillna(subset['avg_chol_by_age_group'].mean())
subset['avg_chol_by_age_group'].isnull().sum()

In [ ]:
# ESTO PUEDE JUNTARSE EN DOS CELDAS, UNA CON LAS NUMERICAS Y OTRA CON LAS CUALITATIVAS

### Variable de salida
**num** 

In [ ]:
subset.num.value_counts().plot.bar(title='num')

In [ ]:
subset.num.value_counts()

In [ ]:
len(subset)

Podemos observar en el grafico que de la variable **num** se encuentra dividida en los siguientes porcentajes:
- **0 - 55,21%**
- **1 - 18.51%**
- **2 - 11.78%**
- **3 - 11.78%**
- **4 - 4.37%**

Esto nos indica que el 55% de las personas no se detectaron enfermedades cardiacas, y el resto, el 45% indica que hay enfermedad cardiaca entre sus diferentes gravedades. Esto nos muestra que hay una tendencia en la cual mientras más grave sea la enfermedad cardiaca, menor es la cantidad de personas hay.

In [ ]:
mean = subset['num'].mean()

std_dev = subset['num'].std()

print(f"Media: {mean}")
print(f"Desviación estándar: {std_dev}")

Interpretación de la Media: Es 0.9337, lo que indica que el valor promedio de la variable es cercano a 1. Esto sugiere que la mayoría de los datos están en los valores más bajos (0 y 1), con algunos valores más altos contribuyendo a elevarla ligeramente.

Interpretación de la Desviación Estándar: Es 1.229, lo que nos indica cuánto se desvían los datos de la media, en promedio. Es relativamente alta en comparación con la media, lo que sugiere que hay una considerable variabilidad en los datos con una distribución sesgada hacia los valores más bajos (cercanos a 0). Esto es consistente con una distribución en la que hay pocos datos con valores altos (3 o 4), lo cual tiene sentido, dado que es más complejo tener mayor cantidad de registros de pacientes con mayor gravedad.

### Variables de entrada

Para trabajar de una manera mas entendible en el trabajo decidimos renombrar las mismas.

In [ ]:
BETTER_COLUMN_NAMES = {
    'sex': 'sex',
    'cp': 'chest_pain',
    'fbs':'fasting_blood_sugar',
    'restecg':'rest_ecg',
    'thalach':'max_heart_rate',
    'exang':'exercise_induced_angina',
    'oldpeak':'depression_induced_ex',
    'slope':'slope',
    'ca':'vessels_colored_fl',
    'thal':'thalassemia',
    'num':'diagnosis',
    'age_group':'age_group',
    'cholesterol_level':'cholesterol_level',
    'bp_level':'blood_pressure_level',
    'risk_score':'risk_score',
    'symptom_severity':'symptom_severity',
    'risk_factor':'risk_factor',
    'avg_chol_by_age_group':'avg_chol_by_age_group'
}

subset.rename(columns=BETTER_COLUMN_NAMES, inplace=True)

subset

**sex**

In [ ]:
subset.sex.value_counts()

In [ ]:
subset.sex.value_counts().plot.bar(title='Sex (1: Hombre 0: Mujer)')

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'sex']).size().unstack(fill_value=0)
grouped_data.plot(kind='bar', title='Distribución de sex por diagnosis')

plt.xlabel('diagnosis')
plt.ylabel('Cantidad')
plt.show()

Hay que considerar que los registros de sex se encuentra desbalaceados, siendo la cantidad de hombres en los registros de un 69.02% y de las mujeres el 32.65%, por lo que es evidente que el gráfico indique que los hombres son más propensos a contraer una enfermedad cardiaca. 

**Depression_induced_ex**

In [ ]:
plt.hist(subset.depression_induced_ex, bins=15, edgecolor='black')
plt.show()

In [ ]:
fig = px.box(subset, y='depression_induced_ex')
fig.show()

La variable representa la depresión inducida por el ejercicio en relación al reposo, lo que quiere decir que cuanto mas cercano a 0 sea mejor. Esto es así porque un corazón sano no debería esforzarse tanto para hacer ejercicio.

Podemos observar en el gráfico que, el 50% del total de los datos se encuentran entre 0 y 1,6.

Pero también podemos observar que hay valores aberrantes. A los mismos procedemos a dejarlos por las dudas, dado que el ámbito en el cual estamos trabajando es analizar que tan enferma está una persona del corazón, y estos casos aislados pueden darle mucha información al modelo.

In [ ]:
subset.boxplot(column='depression_induced_ex', by='diagnosis', grid=False)

plt.title('Distribución de depression_induced_ex por diagnosis')
plt.suptitle('')  
plt.xlabel('diagnosis')
plt.ylabel('depression_induced_ex')
plt.show() 

Las estadísticas muestran que la depresión inducida por el ejercicio aumenta a medida que el grupo de diagnóstico sube de 0 a 4. Los grupos con diagnósticos más altos (3 y 4) tienen valores promedio y desviaciones estándar más altos, indicando una mayor intensidad y variabilidad de la depresión inducida. En contraste, los grupos con diagnósticos más bajos (0 y 1) muestran menor intensidad y dispersión. Por lo tanto, hay una relación directamente proporcional entre el diagnóstico y la depresión inducida, con mayor severidad y variabilidad en los diagnósticos más altos.

#### Grupo Etáreo

In [ ]:
subset.age_group.value_counts().plot.bar(title='Grupo etario', )

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'age_group']).size().unstack(fill_value=0)
graph = grouped_data.plot(kind='bar', title='Distribución de age_group por num')

for container in graph.containers:
    graph.bar_label(container, label_type='edge')

plt.xlabel('diagnosis')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
subset.age_group.value_counts()

A simple vista observamos que el grupo etáreo de los **50s** es aquel que más destaca entre los diferentes valores de diagnosis, al igual que aquel grupo que se encuentra entre los **60s**.

**Porcentajes**
- **30s - 4.71%**
- **40s - 24.24%**
- **50s - 41.07%**
- **60s - 26.59%**
- **70s - 3.36%**

Con estos porcentajes observamos que los registros de las edades etarias se encuentra desbalanceados, siendo el grupo etáreo de 50s con el mayor porcentaje. 

Para hacer un análisis más profundo, sacamos la proporción de cada grupo etáreo en funcion con la gravedad de la enfermedad:

30s - Tienen un 71% en 0, un 14% en 1, un 0% en 2, un 7% en 3 y un 7% en 4. 

40s - Tienen un 69% en 0, un 15% en 1, un 7% en 2, un 8% en 3 y un 0% en 4. 

50s - Tienen un 52% en 0, un 19% en 1, un 11% en 2, un 13% en 3 y un 4% en 4. 

60s - Tienen un 40% en 0, un 21% en 1, un 20% en 2, un 13% en 3 y un 6% en 4. 

70s - Tienen un 60% en 0, un 10% en 1, un 0% en 2, un 10% en 3 y un 20% en 4. 

La distribución muestra que la gravedad de la enfermedad aumenta con la edad, especialmente a partir de los 50 años, donde se observa un mayor porcentaje de casos en niveles de gravedad moderada y severa. Los grupos más jóvenes (30s y 40s) presentan una mayoría de casos leves o sin enfermedad, mientras que en los grupos mayores (60s y 70s) se ve una mayor dispersión, con algunos individuos en niveles graves, especialmente en los 70s, donde un 20% alcanza el nivel más alto de severidad. Esto sugiere una relación clara entre la edad y la progresión de la enfermedad.


#### cp - Tipo de dolor de pecho

Es especialmente relevante en un contexto cardíaco, ya que el dolor en el pecho suele estar relacionado con el corazón. Esta clasificación es crucial para evaluar y diferenciar entre diferentes presentaciones clínicas que podrían indicar la presencia o ausencia de una condición cardíaca. 

Valores que puede tomar:

- 1: Angina típica -- suele estar relacionada con problemas coronarios significativos, por lo que es grave.

- 2: Angina atípica -- puede retrasar el tratamiento dado que es díficl de analizar, aunque en algunos casos puede no ser tan grave como la angina típica.

- 3: Dolor no anginal -- este tipo de dolor no suele estar relacionado con el corazón, lo que lo hace menos grave desde el punto de vista cardíaco.

- 4: Asintomático -- al no haber dolor ni otras señales de advertencia, el problema puede pasar desapercibido y no tratarse a tiempo, lo que aumenta el riesgo de complicaciones como infartos en pacientes que estan enfermos. Pero también aquellos que estén sanos entran en esta categoría.

Según lo investigado, el asíntomatico puede ser bueno o muy grave, dado que no asegura que una persona no está enferma, ya que puede desarrollarse una enfermedad cardíaca sin presentar síntomas. 

La pregunta que haríamos es: ¿cómo tomamos los registros asintomáticos, cómo algo bueno o malo? Es decir, ¿cuánto mayor es el valor de estos resultados, menos probabilidades de tener una enfermedad grave? ¿O están desordenados?

In [ ]:
subset.chest_pain.value_counts().plot.bar(title='Tipo de dolor de pecho')

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'chest_pain']).size().unstack(fill_value=0)
graph = grouped_data.plot(kind='bar', title='Distribución de chest_pain por diagnosis')

for container in graph.containers:
    graph.bar_label(container, label_type='edge')

In [ ]:
subset.chest_pain.value_counts()

A simple vista se puede observar que no importa el tipo de dolor de pecho para poder deducir si hay alguna enfermedad cardiaca. Se ve en el gráfico que aquellos pacientes asintomáticos (4) se encuentran distribuidos entre todas las variables del diagnosis. 

**Porcentaje de chest_pain**
- **4 - 46.80%**
- **3 - 28.95%**
- **2 - 16.83%**
- **1 - 7.40%**

#### fbs - Azúcar en sangre en ayunas > 120 mg/dl

In [ ]:
subset.fasting_blood_sugar.value_counts().plot.bar(title='Azúcar en sangre > 120ml/dl')

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'fasting_blood_sugar']).size().unstack(fill_value=0)
graph = grouped_data.plot(kind='bar', title='Distribución de fasting_blood_sugar por diagnosis')

for container in graph.containers:
    graph.bar_label(container, label_type='edge')

plt.xlabel('fasting_blood_sugar')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
subset.fasting_blood_sugar.value_counts()

#### restecg

In [ ]:
subset.rest_ecg.value_counts().plot.bar(title='rest_ecg')

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'rest_ecg']).size().unstack(fill_value=0)
graph = grouped_data.plot(kind='bar', title='Distribución de rest_ecg por diagnosis')

for container in graph.containers:
    graph.bar_label(container, label_type='edge')

plt.xlabel('rest_ecg')
plt.ylabel('Cantidad')
plt.show()

Se observa que aumenta la gravedad de la enfermedad cardiaca (diagnosis) es mayor la hipertrofia ventricular izquierda probable o definitiva (rest_ecg - 2), esto nos dice que es más proble de contraer una enfermedad cardiaca grave. 

#### exang - angina inducida por el ejercicio

In [ ]:
subset.exercise_induced_angina.value_counts().plot.bar(title='exang')

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'exercise_induced_angina']).size().unstack(fill_value=0)
grapf = grouped_data.plot(kind='bar', title='Distribución de xercise_induced_angina por diagnosis')

for container in graph.containers:
    graph.bar_label(container, label_type='edge')

plt.xlabel('exercise_induced_angina')
plt.ylabel('Cantidad')
plt.show()

#### slope

In [ ]:
subset.slope.value_counts().plot.bar(title='slope')

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'slope']).size().unstack(fill_value=0)
graph = grouped_data.plot(kind='bar', title='Distribución de slope por diagnosis')

for container in graph.containers:
    graph.bar_label(container, label_type='edge')

plt.xlabel('slope')
plt.ylabel('Cantidad')
plt.show()

El ST ascendente (valor 1 del **slope**) es el valor menos procupante a la hora de tener que indicar alguna enfermedad cardiaca por lo que corresponde con lo que nos indica el gráfico, y a medica que aumenta la posibilidad de tener una enfermedad cardiaca, este valor disminuye y se muestra predominante el valor de ST plano (valor 2 de **slope**). 

#### Vessels colored fl

In [ ]:
subset.vessels_colored_fl.sample(10)

Aca tenemos un problema, el tipo de variable de 'vessels_colored_fl' es float, pero los datos adentro son todos enteros, asi que seran transformados en int.

In [ ]:
subset['vessels_colored_fl'] = subset['vessels_colored_fl'].astype(int)

In [ ]:
subset.vessels_colored_fl.value_counts().plot.bar()

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'vessels_colored_fl']).size().unstack(fill_value=0)
graph = grouped_data.plot(kind='bar', title='Distribución de vessels_colored_fl por diagnosis')

for container in graph.containers:
    graph.bar_label(container, label_type='edge')

plt.xlabel('diagnosis')
plt.ylabel('Cantidad')
plt.show()

Se evidencia que aquellos que no poseen enfermedad cardiaca, son aquellos que no poseen vasos principales bloqueados. Por lo que a medida que aumenta la gravedad de la enfermedad cardiaca, disminuye. 

#### Thalassemia

Los valores de "thalassemia" son enteros, solo que en el dataframe se encuentran almacenados como float, entonces a la hora de realizar el gráfico aparecen un par de errores. 

In [ ]:
subset["thalassemia"] = subset["thalassemia"].astype(int)

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'thalassemia']).size().unstack(fill_value=0)
graph = grouped_data.plot(kind='bar', title='Distribución de thalassemia por diagnosis')

for container in graph.containers:
    graph.bar_label(container, label_type='edge')

plt.xlabel('diagnosis')
plt.ylabel('Cantidad')
plt.show()

#### Max_heart_rate

In [ ]:
subset.boxplot(column='max_heart_rate', by='diagnosis', grid=False)

plt.title('Distribución de max_heart_rate por diagnosis')
plt.suptitle('')
plt.xlabel('diagnosis')
plt.ylabel('thalassemia')
plt.show()

#### Cholesterol_level

In [ ]:
subset.cholesterol_level.value_counts().plot.bar()

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'cholesterol_level']).size().unstack(fill_value=0)
graph = grouped_data.plot(kind='bar', title='Distribución de cholesterol_level por diagnosis')

for container in graph.containers:
    graph.bar_label(container, label_type='edge')

plt.xlabel('diagnosis')
plt.ylabel('Cantidad')
plt.show()

#### Blood_pressure_level

In [ ]:
grouped_data = subset.groupby(['diagnosis', 'blood_pressure_level']).size().unstack(fill_value=0)
graph = grouped_data.plot(kind='bar', title='Distribución de blood_pressure_level por diagnosis')

for container in graph.containers:
    graph.bar_label(container, label_type='edge')

plt.xlabel('diagnosis')
plt.ylabel('Cantidad')
plt.show()

#### Risk_score

In [ ]:
fig = px.box(subset, y='risk_score')
fig.show()

In [ ]:
bxp = subset.boxplot(column='risk_score', by='diagnosis', grid=False)

plt.title('Distribución de risk_score por diagnosis')
plt.suptitle('')
plt.xlabel('diagnosis')
plt.ylabel('risk_score')
plt.show()

#### Risk_factor

In [ ]:
fig = px.box(subset, y='risk_factor')
fig.show()

In [ ]:
bxp = subset.boxplot(column='risk_factor', by='diagnosis', grid=False)

plt.title('Distribución de risk_factor por diagnosis')
plt.suptitle('')
plt.xlabel('diagnosis')
plt.ylabel('risk_factor')
plt.show()

#### Symptom_severity

In [ ]:
fig = px.box(subset, y='symptom_severity')
fig.show()

In [ ]:
bxp = subset.boxplot(column='symptom_severity', by='diagnosis', grid=False)

plt.title('Distribución de symptom_severity por diagnosis')
plt.suptitle('')
plt.xlabel('diagnosis')
plt.ylabel('symptom_severity')
plt.show()

#### Avg_chol_by_age_group

In [ ]:
fig = px.box(subset, y='avg_chol_by_age_group')
fig.show()

In [ ]:
bxp = subset.boxplot(column='avg_chol_by_age_group', by='diagnosis', grid=False)

plt.title('Distribución de avg_chol_by_age_group por diagnosis')
plt.suptitle('')
plt.xlabel('diagnosis')
plt.ylabel('avg_chol_by_age_group')
plt.show()

In [ ]:
valor_unicos_age_group = subset['age_group'].unique()
vu_chol =  subset['cholesterol_level'].unique()
vu_bp = subset['blood_pressure_level'].unique()

In [ ]:
subset["age_group"] = subset.age_group.replace(valor_unicos_age_group, [60, 30, 40, 50, 70])
subset['cholesterol_level'] = subset.cholesterol_level.replace(vu_chol, [1, 2, 0])
subset['blood_pressure_level'] = subset.blood_pressure_level.replace(vu_bp, [2,0,1])
sns.heatmap(subset.corr(), annot=True, cmap='RdYlGn', linewidths=0.2)
fig = plt.gcf()
fig.set_size_inches(15, 12)
plt.show()

**Menor relacion con diagnosis**
1. **Max_heart_rate** tiene una baja correlacion con **diagnosis**, que es nuestra variable de salida. Esto nos lleva a entender que max_heart_rate no afecta a la hora de determinar el diagnostico. Además podemos ver que **Max_heart_rate** tiene una baja relación con el resto de variables por lo que nos lleva a pensar de que se pueda eleminar del dataset. 

**Estas son las variables con mayor correlacion con diagnosis**
Consideramos que empieza haber cierta correlacion a partir de 0.40.

1. **chest_pain** 0.4
2. **exercise_induced_angina** 0.44
3. **depression_induced_ex** 0.43
4. **vessel_coloured_fl** 0.46
5. **thalassemia** 0.53
6. **symptom_severity** 0.49
7. **risk_factor** 0.54

**Una vez identificadas estas variables, queremos saber con que otras variables poseen una mayor correlación aparte de diagnosis**
1. **chest_pain**
- risk_factor - 0.42
- symptom_severity - 0.42

2. **exercise_induced_angina**
- risk_factor - 0.41

3. **depression_induced_ex**
- slope - 0.54
- symptom_severity - 0.94
- risk_factor - 0.84

5. **thalassemia**
- risk_factor - 0.54

6. **symptom_severity**
- risk_factor - 0.93
- slope - 0.51
- depression_induced_ex - 0.94
- chest_pain - 0.42

7. **risk_factor**
- symptom_severity - 0.93
- chest_pain - 0.42
- exercise_induced_angina - 0.41
- depression_induced_ex - 0.84
- thalassemia - 0.54

Para el resto de variables que no muestran algun signo de generar una alta influencia en diagnosis, se van a probar con eliminarlas del dataset y evaluar si influye considerablemente en el resultado. 

### Listado de posibles dudas/preguntas al encargado de proveer los datos

1. ¿Que tan fiable son los datos?
2. ¿Por qué hay tantos outliners en las siguientes variables risk_score, risk_factor, symptom_severity y avg_chol_by_age_group? 

## 3- Hipótesis sobre los datos

### a. Formulación de hipótesis sobre la variable target bajo determinadas condiciones

1. En la variable **chest_pain** se piensa que esta puede no influir a la hora de predecir si hay o no alguna enfermedad cardiaca.
2. En la variable **fasting_blood_sugar** se piensa que puede no influir a la hora de predecir si hay o no alguna enfermedad cardiaca. No se termina de saber si al ser true nos asegure que pueda haber algun tipo de enfemerdad cardiaca. 
3. Planteamos que las variables mencionadas como aquellas con mayor relación son las que más van a influir con el resultado final de diagnosis y a su vez, cada una de estas variables estan relacionadas con otras variables que se consideraran importantes a la hora de obtener el diagnosis.
4. **Max_heart_rate** no tiene una correlación con diagnosis, por lo que esta variable no tiene importancia para diagnosis.
 

### b. Comprobación de la hipótesis

### c. Creación de nuevas variables

## 4- Modelado

Dividimos el conjunto de datos en train, validation y test

In [ ]:
from sklearn.model_selection import train_test_split

train, not_train = train_test_split(subset, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train.shape, validation.shape, test.shape

Generamos nuestro mapper

In [ ]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler, OneHotEncoder

mapper = DataFrameMapper([
    (['sex'], [OneHotEncoder()]),
    (['chest_pain'], [StandardScaler()]),
    (['fasting_blood_sugar'], [OneHotEncoder()]),
    (['rest_ecg'], [StandardScaler()]),
    (['max_heart_rate'], [StandardScaler()]),
    (['exercise_induced_angina'], [OneHotEncoder()]),
    (['depression_induced_ex'], [StandardScaler()]),
    (['slope'], [StandardScaler()]),
    (['vessels_colored_fl'], [StandardScaler()]),
    (['thalassemia'], [StandardScaler()]),
    (['age_group'], [OneHotEncoder()]),
    (['cholesterol_level'], [OneHotEncoder()]),
    (['blood_pressure_level'], [OneHotEncoder()]),
    (['risk_score'], [StandardScaler()]),
    (['symptom_severity'], [StandardScaler()]),
    (['risk_factor'], [StandardScaler()]),
    (['avg_chol_by_age_group'], [StandardScaler()])
])

mapper.fit(train)

Vemos como realiza las transformaciones

In [ ]:
sample = subset.sample(5, random_state=42)

mapper.transform(sample)

In [ ]:
mapper.transformed_names_

Generamos el Pipeline

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline

pipeLR = Pipeline([
    ('mapper', mapper),
    ('classifier', LogisticRegression(random_state=42))
])
# Lo entrenamos con train
pipeLR.fit(train, train.diagnosis)

y_predLR = pipeLR.predict(validation)

y_predLR

Ahora se evaluarán las metricas

In [ ]:
from sklearn import metrics

print(metrics.classification_report(validation.diagnosis, y_predLR))

Claramente no parece ser un problema para resolver con logisitc regression.

Probaremos con k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

K = 10

pipeKNN = Pipeline([
    ('mapper', mapper),
    ('classifier', KNeighborsClassifier(n_neighbors=K))
])
# Lo entrenamos con train
pipeKNN.fit(train, train.diagnosis)

y_predKNN = pipeKNN.predict(validation)

y_predKNN

In [ ]:
print(metrics.classification_report(validation.diagnosis, y_predKNN))

Modificaremos el valor devuelto por 'diagnosis', de manera que tendremos solo dos resultados posibles [0,1]
0: sin enfermedad
1: con enfermedad

In [ ]:
subset['diagnosis'] = subset['diagnosis'].apply(lambda x: 1 if x > 1 else x)

subset.diagnosis

Realiazamos la sepracion de los datos

In [ ]:
train, not_train = train_test_split(subset, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

Generamos la nueva Pipeline con LogisticRegression y k-NN para comparar

In [ ]:
pipeLR = Pipeline([
    ('mapper', mapper),
    ('classifier', LogisticRegression(random_state=42))
])
# Lo entrenamos con train
pipeLR.fit(train, train.diagnosis)

y_predLR_train = pipeLR.predict(train)
y_predLR_validation = pipeLR.predict(validation)

y_predLR_validation

Evaluamos las metricas

In [ ]:
print(metrics.classification_report(validation.diagnosis, y_predLR_validation))

In [ ]:
cm = metrics.confusion_matrix(train.diagnosis, y_predLR_train)
cm_plot = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                        display_labels=['no disease', 'disease'])
cm_plot.plot(cmap="Blues")
cm_plot.ax_.set_title('train')

In [ ]:
cm = metrics.confusion_matrix(validation.diagnosis, y_predLR_validation)
cm_plot = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                        display_labels=['no disease', 'disease'])
cm_plot.plot(cmap="Blues")
cm_plot.ax_.set_title('validation')

Con los datos de 'diagnosis' modificados, las predicciones son mas precisas.

In [ ]:
K = 10

pipeKNN = Pipeline([
    ('mapper', mapper),
    ('classifier', KNeighborsClassifier(n_neighbors=K))
])
# Lo entrenamos con train
pipeKNN.fit(train, train.diagnosis)

y_predKNN_train = pipeKNN.predict(train)
y_predKNN_validation = pipeKNN.predict(validation)

y_predKNN_validation

In [ ]:
print(metrics.classification_report(validation.diagnosis, y_predKNN_validation))

In [ ]:
cm = metrics.confusion_matrix(train.diagnosis, y_predKNN_train)
cm_plot = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                        display_labels=['no disease', 'disease'])
cm_plot.plot(cmap="Blues")
cm_plot.ax_.set_title('train')

In [ ]:
cm = metrics.confusion_matrix(validation.diagnosis, y_predKNN_validation)
cm_plot = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                        display_labels=['no disease', 'disease'])
cm_plot.plot(cmap="Blues")
cm_plot.ax_.set_title('validation')

Con un KNN con k=10 vemos que es peor que Logistic Regression

In [ ]:
from sklearn.model_selection import GridSearchCV

knn_clf = KNeighborsClassifier(n_neighbors=1)
parameters = {'n_neighbors': [1, 3, 5, 10, 15, 20, 50, 100]}

clf = GridSearchCV(knn_clf, parameters, refit=True, verbose=1)

gs_pipe = Pipeline([
    ('mapper', mapper),
    ('classifier', clf),
])

gs_pipe.fit(train, train.diagnosis)

clf.best_score_, clf.best_params_

In [ ]:
K=15

pipeKNN = Pipeline([
    ('mapper', mapper),
    ('classifier', KNeighborsClassifier(n_neighbors=K))
])

pipeKNN.fit(train, train.diagnosis)

kNN_train = pipeKNN.predict(train)
kNN_validation = pipeKNN.predict(validation)

kNN_validation

In [ ]:
print(metrics.classification_report(validation.diagnosis, kNN_validation))

Una vez evaluado el modelo con Logistic Regression y con k-NN con un k=15 vemos que el Logistic Regression tiene una mayor acuracy [LogisticRegression: 83, k-NN: 78]